## GeoPandas spatial join example

This is a simple example of how to do a spatial join between a data
set of (latitude, longitude) pairs to an Esri shapefile to get the census
tract ID for each lat/lon pair using the Python 
[GeoPandas](http://geopandas.org/)
module without having to use QGIS or ArcPro.

### Python GIS Module Installation

If you're using the Anaconda Python 3 
[standard](https://www.anaconda.com/distribution/) or 
[miniconda](https://docs.conda.io/en/latest/miniconda.html)
distribution, it's easier to get everything working without library conflicts if you
create a separate environment for this GIS work. In the Anaconda Prompt type

```
conda create --name GIS -c conda-forge geopandas fiona shapely
```

then to "activate" this environment, type

```
conda activate GIS
```

### Import modules

In [4]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os

### Read in census tracts shapefile

This is the 2010 California, Los Angeles County shapefile, downloaded from [the US Census Bureau web interface](https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=Census+Tracts)

In [12]:
tracts = gpd.read_file('LA_CensusTracts/CENSUS_TRACTS_2000.shp')

tracts.head()

,CT00,LABEL,Shape_STAr,Shape_STLe,geometry
0,101110,1011.10,1.228537e+07,15056.954289,"POLYGON ((6475581.002 1916939.627, 6475580.499..."
1,101120,1011.20,1.364394e+08,70339.952869,"POLYGON ((6471660.000 1919631.502, 6471661.499..."
2,101210,1012.10,7.019669e+06,12082.404482,"POLYGON ((6473070.498 1913983.375, 6472914.850..."
3,101220,1012.20,7.485860e+06,12652.916695,"POLYGON ((6477593.248 1914091.178, 6477595.001..."
4,101300,1013.00,2.784498e+07,28343.215220,"POLYGON ((6480645.620 1906746.726, 6480594.842..."


### Read in Zipped (.zip) shapefile

If you know the file name and path within a zipped shapefile (set of files in a folder), you don't even have to unzip it before reading!

In [13]:
tracts = gpd.read_file('zip://LA_CensusTracts.zip!/LA_CensusTracts/CENSUS_TRACTS_2000.shp')

tracts.head()

,CT00,LABEL,Shape_STAr,Shape_STLe,geometry
0,101110,1011.10,1.228537e+07,15056.954289,"POLYGON ((6475581.002 1916939.627, 6475580.499..."
1,101120,1011.20,1.364394e+08,70339.952869,"POLYGON ((6471660.000 1919631.502, 6471661.499..."
2,101210,1012.10,7.019669e+06,12082.404482,"POLYGON ((6473070.498 1913983.375, 6472914.850..."
3,101220,1012.20,7.485860e+06,12652.916695,"POLYGON ((6477593.248 1914091.178, 6477595.001..."
4,101300,1013.00,2.784498e+07,28343.215220,"POLYGON ((6480645.620 1906746.726, 6480594.842..."


In [34]:
tracts.crs

{'proj': 'lcc',
 'lat_0': 33.5,
 'lon_0': -118,
 'lat_1': 34.0333333333333,
 'lat_2': 35.4666666666667,
 'x_0': 2000000,
 'y_0': 500000,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

## Read family data

This is a very small, attribute-randomized subset (rows and columns) of household data from [InfoUSA](https://www.infousa.com/). *It will be useless for research use, but works well for this example.*

- Be sure to read 'familyid' as a string or initial zeroes will be lost!
- Only reading in some of the columns for this operation

In [3]:
data_cols = ['familyid','longitude','latitude']
eth = pd.read_csv('la_family_data.csv', sep=',', usecols=data_cols, dtype={'familyid': 'str'})

eth.head()

,familyid,latitude,longitude
0,000106617324,34.13613,-118.14349
1,000106679098,33.84097,-118.06158
2,000106273271,34.69933,-118.13906
3,000106308511,33.79798,-118.11764
4,000106431883,34.40821,-118.44098


### Lat/Lon pairs need to be converted to a Point() data type

We'll put these in a new column called `coordinates`

In [4]:
eth['coordinates'] = list(zip(eth['longitude'], eth['latitude']))
eth['coordinates'] = eth['coordinates'].apply(Point)

eth.head()

,familyid,latitude,longitude,coordinates
0,000106617324,34.13613,-118.14349,POINT (-118.14349 34.13613)
1,000106679098,33.84097,-118.06158,POINT (-118.06158 33.84097)
2,000106273271,34.69933,-118.13906,POINT (-118.13906 34.69933)
3,000106308511,33.79798,-118.11764,POINT (-118.11764 33.79798)
4,000106431883,34.40821,-118.44098,POINT (-118.44098 34.40821)


### Convert to GeoDataFrame, specifying which column contains the geometry

- Have to set the Coordinate Reference System (CRS) initialy to say it's been hand-coded lat/lon, `epsg:4326`
- Then reproject to same CRS as tracts

In [5]:
gdf = gpd.GeoDataFrame(eth, geometry='coordinates')
gdf.crs = {'init': 'epsg:4326'}
gdf = gdf.to_crs(tracts.crs)

### Do the spatial join

- We'll do a "left" join so we don't lose any points if some of our families don't happen to fall within a census tract – we'll just get a NULL (None) in the CT100 field.
- 'within' just slightly faster operation than 'intersects' or 'contains' – For points in a polygon it shouldn't matter which operation you use

In [6]:
eth_tracts = gpd.sjoin(gdf, tracts, how="left", op='within')

eth_tracts[['familyid','CT00']].head()

,familyid,CT00
0,000106617324,463600
1,000106679098,555103
2,000106273271,900806
3,000106308511,574300
4,000106431883,920043


### Saving only the two fields that we need to output CSV

In [7]:
eth_tracts[['familyid','CT00']].to_csv('la_family_tracts.csv', index=False, encoding='utf-8')

### Alternatively save data to shapefile for easier mapping

I like to put shapefiles in their own directory since a "shapefile" really consists of multiple files, and this makes it easier to keep them together.

- *Creating the directory will give an error if it already exists, so I check whether it's already been created first.*
- *This also happens to be an easy way to create a DBF file of your data!*

In [8]:
if not os.path.exists('la_family_SHP'):
	os.mkdir('la_family_SHP')
    
eth_tracts[['familyid', 'coordinates', 'CT00']].to_file('la_family_SHP/la_family_points.shp')

---

## Geo shape centroid

There is a .centroid property of any geometry shapely (GeoPandas) column
https://gis.stackexchange.com/questions/166820/geopandas-return-lat-and-long-of-a-centroid-point

`tracts[‘geometry’].centroid`

and each of those has a `.x` and `.y` attribute (centroid is a `Point()`)

In [39]:
geoms = tracts.loc[:10,['CT00','geometry']].copy()

In [40]:
geoms.crs

{'proj': 'lcc',
 'lat_0': 33.5,
 'lon_0': -118,
 'lat_1': 34.0333333333333,
 'lat_2': 35.4666666666667,
 'x_0': 2000000,
 'y_0': 500000,
 'datum': 'NAD83',
 'units': 'us-ft',
 'no_defs': True}

In [30]:
geoms['geometry'].centroid

0     POINT (6473138.216 1916941.393)
1     POINT (6479969.894 1921132.951)
2     POINT (6473815.754 1914573.486)
3     POINT (6476559.474 1914071.099)
4     POINT (6479779.632 1913100.254)
5     POINT (6472849.530 1910858.903)
6     POINT (6464086.524 1907684.472)
7     POINT (6456608.263 1901932.035)
8     POINT (6469673.720 1924292.753)
9     POINT (6468792.850 1918379.697)
10    POINT (6460368.124 1922514.284)
dtype: geometry

In [32]:
geoms['geometry'].centroid.x

0     6.473138e+06
1     6.479970e+06
2     6.473816e+06
3     6.476559e+06
4     6.479780e+06
5     6.472850e+06
6     6.464087e+06
7     6.456608e+06
8     6.469674e+06
9     6.468793e+06
10    6.460368e+06
dtype: float64

#### Get centroids in lat/lon

In [44]:
geoms.to_crs(crs='epsg:4326', inplace=True)
geoms['geometry'].centroid

0     POINT (-118.29298 34.25948)
1     POINT (-118.27041 34.27105)
2     POINT (-118.29072 34.25298)
3     POINT (-118.28163 34.25162)
4     POINT (-118.27097 34.24898)
5     POINT (-118.29388 34.24277)
6     POINT (-118.32284 34.23397)
7     POINT (-118.34752 34.21809)
8     POINT (-118.30452 34.27965)
9     POINT (-118.30738 34.26340)
10    POINT (-118.33531 34.27468)
dtype: geometry